In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  Y


# Projeto buscas

## 1. Autores:

* Arthur Fernandes de Morais
* João Custódio de Faria Filho
* Raphael de Vasconcelos Nascimento

## 2. Objetivo do Trabalho e descrição da implementação:

O presente trabalho visa a resolver os problemas 1 e 2 descritos abaixo. Para a solução de ambos os problemas foi utilizado a linguagem `Python 3.7` e  implementado via `Juyter Notebooks`. Utilizou-se também, para sincronizar as modificações pelos autores, a plataforma do `github` por meio de um repositório contendo tanto os notebooks quanto os arquivos auxiliares à solução dos problemas que são:

* Para o problema 1: Além do notebook `q1.ipynb`, o arquivo `australia.csv`
* Para o problema 2: Além do notebook `q2.ipynb`, os arquivos `puzzle.sav`, `puzzle2.sav`, `puzzle3.sav` e `puzzle4.sav`

## 3. Resultados obtidos

#### 2.1 Descrição da solução do seguinte problema:
Crie um agente capaz de encontrar o menor caminho entre duas cidades, com
mapa definido como segue. O agente deve receber como entradas o id da cidade
origem, id da cidade destino e nome do arquivo de dados. Usando este agente
encontre o menor caminho entre as cidades Alice Springs (id 5) e Yulara da
Australia(id 219) do arquivo australia.csv, explicite o caminho (a lista das cidades)
da solução e também a distância do início ao fim.

In [18]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
import math
import networkx as nx
import igraph

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

**Para o problema em questão desenvolvemos as seguintes funções:**

`dist_cities_line(city1, city2)`: Calcula a distância em linha reta entre duas cidades considerando a distância euclidiana

`create_graph(data)`: Cria o grafo com as cidades a partir do conjunto de dados disponibilizado

`get_road_distance(origin, destiny, G)`: Retorna a distância entre cidades adjacentes do Grafo

`find_route(origin, destiny, G)`: Retorna a lista de cidades exploradas (em ordem) e a distância total percorrida pelo agente

In [8]:
def dist_cities_line(city1, city2):
    lat_x1 = float(data[data['id'] == city1]['lat'])
    lng_x1 = float(data[data['id'] == city1]['lng'])
    
    lat_x2 = float(data[data['id'] == city2]['lat'])
    lng_x2 = float(data[data['id'] == city2]['lng'])
    
    return math.sqrt((lat_x1 - lat_x2)**2 + (lng_x1 - lng_x2)**2)


def create_graph(data):
    road_factor = 1.1
    
    G = nx.Graph()
    data.apply(lambda x: G.add_node(x['id'], city= x['city']), axis=1)
    
    for city_id in list(G.nodes):
        if city_id > 1 and city_id%2 == 0:
            # Connects with x-1
            G.add_edge(city_id, city_id - 1, distance=road_factor*dist_cities_line(city_id, city_id - 1)) 
            # If exists, connects with x+2
            if city_id < len(data['city']) - 1:
                G.add_edge(city_id, city_id + 2, distance=road_factor*dist_cities_line(city_id, city_id + 2))       
        elif city_id%2 != 0 and city_id > 2:
            # Connects with x-2
            G.add_edge(city_id, city_id - 2, distance=road_factor*dist_cities_line(city_id, city_id - 2))
            # If exists, connects with x+1
            if city_id < len(data['city']):
                G.add_edge(city_id, city_id + 1, distance=road_factor*dist_cities_line(city_id, city_id + 1))
            
    return G

def get_road_distance(origin, destiny, G):
    try:
        distance = G[origin][destiny]["distance"]
    except:
        if origin == destiny:
            distance = 0
        else:
            distance = np.inf
    return distance
    
def find_route(origin, destiny, G):
    # List of all cities in the graph
    cities = list(G.nodes)
    # Calculated cost to go to the city from the origin, aiming the destiny
    arrival_cost = np.inf * np.ones(len(G.nodes))
    arrival_cost[origin-1] = 0
    # List of explored cities, just to avoid percurring the previous list complete
    # every time when is required to identify the explored cities 
    explored = [origin]
    # To reconstruct the route
    predecessor = np.zeros(len(cities))
    
  
    # Atributes the "False" value to "routed", and checks for the trivial case
    routed = False
    if(origin == destiny):
        routed = True 
    
    # Repeats while the route to destiny is not complete
    while (not routed):
        # A new city to explore
        explore = {}
        # Smallest cost to explore a new city until now
        smallest_exp_cost = np.inf
        # Calculate costs in the neighborhood of the explored cities
        for city in explored:
            
            # Just to make the algorithmru faster in this case
            #rang = range(max(1, city-3), min(len(cities)+1, city+3))
            
            #for neighbor in rang:
            for neighbor in G.neighbors(city):
                # Checks only the not explored cities
                if arrival_cost[neighbor-1] == np.inf:
                    # Calculates the expected cost function using the  straight line distance as heuristic                  
                    exp_cost = arrival_cost[city-1] + get_road_distance(city, neighbor, G) + dist_cities_line(neighbor, destiny)
                    # In the case the cost_function is smaller than the smallest until now,
                    # the first should take the latter's place 
                    # print("small:", smallest_exp_cost)
                    if exp_cost < smallest_exp_cost:                        
                        explore = {"city": neighbor, "expCost": exp_cost, "predecessor": city}
                        smallest_exp_cost = exp_cost
        
        # As the frontiers exploration ended, adds the new explored city
        explored.append(explore["city"])
        predecessor[explore["city"] - 1] = explore["predecessor"]
        arrival_cost[explore["city"] - 1] = smallest_exp_cost - dist_cities_line(explore["city"], destiny)       
        
        
        # If the algorithm found the destiny, then the route is done
        if(explore["city"] == destiny):
            routed = True      

    # Reconstructs the route
    start = destiny
    route = [destiny]
    while(start != origin):
        start = int(predecessor[start-1] )
        route.insert(0, start)
    
    return route, arrival_cost[destiny-1]


----
**Inicializando o grafo com as cidades**: Vale ressaltar que na nossa implementação cada nó do grafo é composto por um elemento que possui o id da cidade, a partir do conjunto de dados disponibilizado, e um atribudo com o nome da cidade em questão. Além disso, percebe-se que a busca foi inicializada com uma origem e um destino definidos pelo problema.

In [9]:
origin = 5 #Alice Sprnigs
destiny = 219 # Yulara
data = pd.read_csv('australia.csv')
G = create_graph(data)

----
**Cidades exploradas na busca: Em ordem de exploração**: A partir da lista dos nós do grafo explorados na busca, é possível checar quais cidades foram exploradas, com o auxílio do arquivo `australia.csv`

In [5]:
final_answer = find_route(origin, destiny, G)
cities = list(data['city'])
for c in list(final_answer[0]):
    print(str(cities[c-1]))

Alice Springs
Andamooka
Armidale
Ayr
Ballarat
Bairnsdale East
Ballina
Batemans Bay
Bathurst
Bendigo
Bicheno
Birdsville
Bordertown
Bourke
Brisbane
Broome
Bundaberg
Burnie
Byron Bay
Cairns
Caboolture
Caloundra
Canberra
Ceduna
Charleville
Clare
Cobram
Colac
Cowell
Cranbourne
Dalby
Deniliquin
Dubbo
East Maitland
Eidsvold
Esperance
Forbes
Gawler
Georgetown
Gingin
Geraldton
Gladstone
Goondiwindi
Griffith
Gympie South
Hamilton
Hobart
Hughenden
Inverell
Kalbarri
Karratha
Katanning
Katoomba
Kiama
Kimba
Kingoonya
Kingston South East
Kwinana
Laverton
Leonora
Longreach
Manjimup
Maryborough
Meekatharra
Melton
Melbourne
Meningie
Mildura
Morawa
Mount Barker
Mount Isa
Mudgee
Muswellbrook
Narrabri West
Newcastle
Norseman
North Mackay
North Lismore
North Scottsdale
Nowra
Oatlands
Orange
Pambula
Parkes
Perth
Penola
Peterborough
Port Augusta West
Port Douglas
Port Lincoln
Port Pirie
Portland
Queanbeyan
Quilpie
Richmond
Rockhampton
Roma
Scone
Shepparton
Seymour
Singleton
South Grafton
South Melbourne
Stawe

-----
**Checando se a ordem de exploração do algorítmo desenvolvido confere com o método dijkstra_path da biblioteca networkx**: Utilizamos uma implementação do algorítmo de Dijkstra da biblioteca networkx para checarmos se nosso algorítmo foi implementado de maneira correta. Como é possível verificar, tanto a lista de cidades em ordem de busca `final_answer[0]` quanto o caminho total percorrido `final_answer[1]` estão coerentes com o esperado pelo algorítmo.

In [6]:
path_check = nx.dijkstra_path(G, source=origin, target=destiny, weight="distance")
distancy_check = 0
for i in range(0, len(path_check)-1):
    distancy_check = distancy_check + G[path_check[i]][path_check[i+1]]["distance"]

In [7]:
final_answer[0] == path_check

True

In [8]:
final_answer[1] == distancy_check

True